In [1]:
import os
import glob
import math
import pickle

import numpy as np
import pandas as pd
import geopandas as gpd
import torch

import processing_io as pio
from torch_geometric.transforms import LineGraph

from torch_geometric.data import Data, Batch
import shapely.wkt as wkt


highway_mapping = {
    'trunk': 0, 'trunk_link': 0, 'motorway_link': 0,
    'primary': 1, 'primary_link': 1,
    'secondary': 2, 'secondary_link': 2,
    'tertiary': 3, 'tertiary_link': 3,
    'residential': 4, 'living_street': 5,
    'pedestrian': 6, 'service': 7,
    'construction': 8, 'unclassified': 9,
    'np.nan': -1
}

result_df_name = 'sim_output_1pm_capacity_reduction_10k_PRELIMINARY'

result_path = '../../../../data/datasets_simulation_outputs/' + result_df_name + '.pt'
string_is_for_1pm = "pop_1pm"

base_dir = '../../../../data/' + string_is_for_1pm + '_simulations/' + string_is_for_1pm + '_policies_combinations_with_normal_dist/'
subdirs_pattern = os.path.join(base_dir, 'output_networks_*')
subdirs = list(set(glob.glob(subdirs_pattern)))
subdirs.sort()

gdf_basecase_output_links = gpd.read_file('results/' + string_is_for_1pm + '_basecase_average_output_links.geojson')
gdf_basecase_average_mode_stats = pd.read_csv('results/' + string_is_for_1pm + '_basecase_average_mode_stats.csv', delimiter=';')


## Process results

Process the outputs of the simulations for further usage by GNN.

In [2]:
# Read all network data into a dictionary of GeoDataFrames
result_dic = {}
base_network_no_policies = gdf_basecase_output_links
result_dic["base_network_no_policies"] = base_network_no_policies
counter = 0

# Loop through each subdirectory
for subdir in subdirs:
    print(f'Accessing folder: {subdir}')
    print(len(os.listdir(subdir)))
    networks = [network for network in os.listdir(subdir) if not network.endswith(".DS_Store")]
    for network in networks:
        file_path = os.path.join(subdir, network)
        policy_key = pio.create_policy_key_1pm(network)
        gdf_output_links = pio.read_output_links(file_path)
        if (gdf_output_links is not None):
            gdf_extended = pio.extend_geodataframe(gdf_base=gdf_basecase_output_links, gdf_to_extend=gdf_output_links, column_to_extend='highway', new_column_name='highway')
            gdf_extended = pio.extend_geodataframe(gdf_base=gdf_basecase_output_links, gdf_to_extend=gdf_extended, column_to_extend='vol_car', new_column_name='vol_car_base_case')
            result_dic[policy_key] = gdf_extended

Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_policies_combinations_with_normal_dist/output_networks_100
78
Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_policies_combinations_with_normal_dist/output_networks_1000
98
Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_policies_combinations_with_normal_dist/output_networks_1100
99
Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_policies_combinations_with_normal_dist/output_networks_1200
99
Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_policies_combinations_with_normal_dist/output_networks_1300
100
Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_policies_combinations_with_normal_dist/output_networks_1400
99
Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_policies_combinations_with_normal_dist/output_networks_1500
99
Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_policies_combinations_with_normal_dist/output_networks_

In [3]:
result_dic_mode_stats = {}
result_dic_mode_stats["base_network_no_policies"] = gdf_basecase_average_mode_stats
counter = 0

def calculate_averaged_results(trips_df):
    """Calculate average travel time and routed distance grouped by mode."""
    return trips_df.groupby('mode').agg(
        total_travel_time=('travel_time', 'mean'),
        total_routed_distance=('routed_distance', 'mean')
    ).reset_index()

for subdir in subdirs:
    # print(f'Accessing folder: {subdir}')
    # print(len(os.listdir(subdir)))
    networks = [network for network in os.listdir(subdir) if not network.endswith(".DS_Store")]
    for network in networks:
        file_path = os.path.join(subdir, network)
        policy_key = pio.create_policy_key_1pm(network)
        df_mode_stats = pd.read_csv(file_path + '/eqasim_trips.csv', delimiter=';')
        averaged_results = calculate_averaged_results(df_mode_stats)
        if (averaged_results is not None):
            result_dic_mode_stats[policy_key] = averaged_results


## Analyze results and plot

In [4]:
# pio.analyze_geodataframes(result_dic=result_dic, consider_only_highway_edges=True)

In [5]:
# pio.analyze_geodataframes(result_dic=result_dic, consider_only_highway_edges=False)

In [6]:
def process_result_dic(result_dic, result_dic_mode_stats):
    datalist = []
    linegraph_transformation = LineGraph()
    base_network_no_policies = result_dic.get("base_network_no_policies")
    vol_base_case = base_network_no_policies['vol_car'].values
    capacity_base_case = base_network_no_policies['capacity'].values
    
    # Initialize base edge positions
    gdf_base = gpd.GeoDataFrame(base_network_no_policies, geometry='geometry')
    gdf_base.crs = "EPSG:2154"  # Assuming the original CRS is EPSG:2154
    gdf_base.to_crs("EPSG:4326", inplace=True)
    edge_positions_base = np.array([((geom.coords[0][0] + geom.coords[-1][0]) / 2, 
                                     (geom.coords[0][1] + geom.coords[-1][1]) / 2) 
                                    for geom in gdf_base.geometry])
    
    nodes = pd.concat([gdf_base['from_node'], gdf_base['to_node']]).unique()
    node_to_idx = {node: idx for idx, node in enumerate(nodes)}
    gdf_base['from_idx'] = gdf_base['from_node'].map(node_to_idx)
    gdf_base['to_idx'] = gdf_base['to_node'].map(node_to_idx)
    edges_base = gdf_base[['from_idx', 'to_idx']].values
    edge_positions_tensor = torch.tensor(edge_positions_base, dtype=torch.float)
    edge_index = torch.tensor(edges_base, dtype=torch.long).t().contiguous()
    x = torch.zeros((len(nodes), 1), dtype=torch.float)
    data = Data(edge_index=edge_index, x=x, pos=edge_positions_tensor)
    linegraph_data = linegraph_transformation(data)

    for key, df in result_dic.items():        
        if isinstance(df, pd.DataFrame) and key != "base_network_no_policies":
            gdf = gpd.GeoDataFrame(df, geometry='geometry')
            gdf.crs = "EPSG:2154"  # Assuming the original CRS is EPSG:2154
            gdf.to_crs("EPSG:4326", inplace=True)

            capacities_new = gdf['capacity'].values
            capacity_reduction = capacities_new - capacity_base_case
            
            highway = gdf['highway'].apply(lambda x: highway_mapping.get(x, -1)).values
            edge_car_volume_difference = gdf['vol_car'].values - vol_base_case
            target_values = torch.tensor(edge_car_volume_difference, dtype=torch.float).unsqueeze(1)

            linegraph_x = torch.tensor(np.column_stack((vol_base_case, capacity_base_case, capacities_new, capacity_reduction, highway)), dtype=torch.float)
            linegraph_data.x = linegraph_x
            linegraph_data.y = target_values
            
            df_mode_stats = result_dic_mode_stats.get(key)
            if df_mode_stats is not None:
                # Convert numeric columns to float
                numeric_cols = df_mode_stats.select_dtypes(include=[np.number]).columns
                mode_stats_numeric = df_mode_stats[numeric_cols].astype(float)
                
                # Convert to tensor
                mode_stats_tensor = torch.tensor(mode_stats_numeric.values, dtype=torch.float)
                
                # Add mode_stats as an attribute to linegraph_data
                linegraph_data.mode_stats = mode_stats_tensor
                
            if linegraph_data.validate(raise_on_error=True):
                datalist.append(linegraph_data)
            else:
                print("Invalid line graph data")
    data_dict_list = [{'x': lg_data.x, 'edge_index': lg_data.edge_index, 'pos': lg_data.pos, 'y': lg_data.y, 'graph_attr': lg_data.mode_stats} for lg_data in datalist]
    return data_dict_list

data_processed = process_result_dic(result_dic=result_dic, result_dic_mode_stats=result_dic_mode_stats)
torch.save(data_processed, result_path)

In [1]:
data_processed[0]

NameError: name 'data_processed' is not defined

## Save for further processing with GNN

In [8]:
# data_processed_single_districts = pio.process_result_dic(result_dic_single_districts)
# torch.save(data_processed_single_districts, result_path + '_single_districts.pt')

In [9]:
torch.save(data_processed, result_path)